### trade based simulation

# data import using yfinance

## 라이브러리 임포트

In [ ]:
!pip install yfinance
!pip install pandas-datareader

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import math
import yfinance as yf
import pandas_datareader as pdr

In [4]:
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

## 데이터 임포트

In [7]:
tickers = ["SPY", "QQQ", "VTWO"]

In [8]:
divs = []
for ticker in tickers:
  div = yf.Ticker(ticker).dividends
  div.index = pd.to_datetime(div.index).strftime('%Y-%m')
  if not div.index.is_unique:
    div = div.groupby(div.index).sum()
  divs.append(div)

In [9]:
dividends_df = pd.concat(divs, axis=1)
dividends_df.columns = [["Dividend"] * len(tickers), tickers]
dividends_df.index.name = "Date"
dividends_df.sort_index(inplace=True)

In [10]:
tickers_df = yf.download(" ".join(tickers), period='max', interval='1mo')

[*********************100%***********************]  3 of 3 completed


In [11]:
tickers_df.index = pd.to_datetime(tickers_df.index).strftime('%Y-%m')

In [12]:
tickers_df.drop(columns=["Adj Close", "High", "Low", "Open", "Volume"], inplace=True)
tickers_df.dropna(how="all", inplace=True)

In [13]:
tickers_df = tickers_df.reindex(columns=tickers, level=1)

In [14]:
if not tickers_df.index.is_unique:
    tickers_df = tickers_df.groupby(tickers_df.index).last()

In [15]:
tickers_df

Close                       
                SPY         QQQ       VTWO
Date                                      
1993-02   44.406250         NaN        NaN
1993-03   45.187500         NaN        NaN
1993-04   44.031250         NaN        NaN
1993-05   45.218750         NaN        NaN
1993-06   45.062500         NaN        NaN
...             ...         ...        ...
2021-09  429.140015  357.959991  88.410004
2021-10  459.250000  386.109985  92.190002
2021-11  455.559998  393.820007  88.360001
2021-12  474.959991  397.850006  89.809998
2022-01  477.095001  394.520111  90.930000

[348 rows x 3 columns]

In [16]:
dividends_df

Dividend               
             SPY    QQQ    VTWO
Date                           
1993-03    0.213    NaN     NaN
1993-06    0.318    NaN     NaN
1993-09    0.286    NaN     NaN
1993-12    0.317    NaN     NaN
1994-03    0.271    NaN     NaN
...          ...    ...     ...
2020-12    1.580  0.561  0.3400
2021-03    1.278  0.395  0.1525
2021-06    1.376  0.397  0.1690
2021-09    1.428  0.414  0.2210
2021-12    1.633  0.491  0.4750

[118 rows x 3 columns]

In [17]:
tickers_df = pd.concat([tickers_df, dividends_df], axis=1)

In [18]:
tickers_df.columns = [["Close"] * len(tickers) + ["Dividend"] * len(tickers), tickers * 2]

In [25]:
tickers_df.index[0], tickers_df.index[-1]

('1993-02', '2022-01')

In [33]:
tb3ms_df = pdr.DataReader("TB3MS", "fred", tickers_df.index[0], tickers_df.index[-1] )

In [34]:
tb3ms_df

,TB3MS
DATE,
1993-02-01,2.93
1993-03-01,2.95
1993-04-01,2.87
1993-05-01,2.96
1993-06-01,3.07
...,...
2021-08-01,0.05
2021-09-01,0.04
2021-10-01,0.05


# backtesting

In [32]:
tickers_df

Close                        Dividend              
                SPY         QQQ       VTWO      SPY    QQQ   VTWO
1993-02   44.406250         NaN        NaN      NaN    NaN    NaN
1993-03   45.187500         NaN        NaN    0.213    NaN    NaN
1993-04   44.031250         NaN        NaN      NaN    NaN    NaN
1993-05   45.218750         NaN        NaN      NaN    NaN    NaN
1993-06   45.062500         NaN        NaN    0.318    NaN    NaN
...             ...         ...        ...      ...    ...    ...
2021-09  429.140015  357.959991  88.410004    1.428  0.414  0.221
2021-10  459.250000  386.109985  92.190002      NaN    NaN    NaN
2021-11  455.559998  393.820007  88.360001      NaN    NaN    NaN
2021-12  474.959991  397.850006  89.809998    1.633  0.491  0.475
2022-01  477.095001  394.520111  90.930000      NaN    NaN    NaN

[348 rows x 6 columns]